In [8]:
import requests

args = ("q=Obama",
        "begin_date='20180101'")
args2 = ('q=trump+women+accuse',
        'begin_date=20151001')

def makeURL(args):
    url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?'
    for i in args:
        url += '&'+i
    url += "&api-key=6a6c043c723548a98eafc5b624317cac"
    return(url)

request_url = makeURL(args2)
print(request_url)
r = requests.get(request_url)
data = r.json()


http://api.nytimes.com/svc/search/v2/articlesearch.json?&q=trump+women+accuse&begin_date=20151001&api-key=6a6c043c723548a98eafc5b624317cac


In [9]:
print(data)

{'status': 'OK', 'copyright': 'Copyright (c) 2018 The New York Times Company. All Rights Reserved.', 'response': {'docs': [{'web_url': 'https://www.nytimes.com/2018/10/19/us/politics/golden-handcuffs-book-trump-women.html', 'snippet': '“Golden Handcuffs: The Secret History of Trump’s Women” traces how the president’s consequence-free (so far) comments about women continue a running theme that started early in his life.', 'print_page': '13', 'blog': {}, 'source': 'The New York Times', 'multimedia': [{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'images/2018/10/20/us/politics/20dc-trumpwomen/merlin_145489149_6a382279-3f13-46ec-82fd-a55d93102c60-articleLarge.jpg', 'height': 400, 'width': 600, 'legacy': {'xlarge': 'images/2018/10/20/us/politics/20dc-trumpwomen/merlin_145489149_6a382279-3f13-46ec-82fd-a55d93102c60-articleLarge.jpg', 'xlargewidth': 600, 'xlargeheight': 400}, 'subType': 'xlarge', 'crop_name': 'articleLarge'}, {'rank': 0, 'subtype': 

In [18]:
num_pages = 10

args2 = ('q=trump+women+accuse',
        'begin_date=20151001')
request_url = makeURL(args2)

pages_all = []
for x in range(num_pages):
    page_request_url = request_url+'page='+str(x)
    r = requests.get(page_request_url)
    news = parse_articles(r)
    pages_all = pages_all.append(news)


TypeError: 'Response' object is not subscriptable

In [ ]:
import json
import time
import pandas as pd

api = articleAPI('6a6c043c723548a98eafc5b624317cac')

data = pd.DataFrame()
for x in range(10):
#    f = api.search(q='Metis', begin_date='20000101', page=str(x))
    f = api.search(q = 'Obama',fq='Obama', begin_date=20180101)
    #print(f.keys())
    #print(f['response']['docs'])

    try:
        print('worked')
        data = data.append(f['response']['docs'])
        time.sleep(1)
    except:
        print('pass')
        pass

with open('results.json', 'w') as fp:
    json.dump(data, fp)

In [17]:
def parse_articles(articles):
    '''
    This function takes in a response to the NYT api and parses
    the articles into a list of dictionaries
    '''
    news = []
    for i in articles['response']['docs']:
        dic = {}
        dic['id'] = i['_id']
        if 'abstract' in i:
            dic['abstract'] = i['abstract'].encode("utf8")
        dic['headline'] = i['headline']['main'].encode("utf8")
        dic['desk'] = i['news_desk']
        dic['date'] = i['pub_date'][0:10] # cutting time of day.
        if 'section_name' in i:
            dic['section'] = i['section_name']
        if 'snippet' in i:
            dic['snippet'] = i['snippet'].encode("utf8")
        dic['source'] = i['source']
        dic['type'] = i['type_of_material']
        dic['url'] = i['web_url']
        dic['word_count'] = i['word_count']
        # locations
        locations = []
        for x in range(0,len(i['keywords'])):
            if 'glocations' in i['keywords'][x]['name']:
                locations.append(i['keywords'][x]['value'])
        dic['locations'] = locations
        # subject
        subjects = []
        for x in range(0,len(i['keywords'])):
            if 'subject' in i['keywords'][x]['name']:
                subjects.append(i['keywords'][x]['value'])
        dic['subjects'] = subjects   
        news.append(dic)
    return(news) 